In [2]:
import tensorflow as tf
import keras
import numpy as np
#import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import layers, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.models import load_model
import os, shutil, pathlib
from PIL import Image
import json

np.random.seed(21)

In [ ]:
# global_best_model.keras download (642 MB):
# https://mega.nz/file/tMp2lQbD#upOPcZM5IR_Rfzts0c9kiCd6qWgwdH6BwHTt2a7CT_4